In [1]:
# pipenv install pandas scipy plotly scikit-learn optuna shap ipykernel ipywidgets nbformat numpy==2.0

# Importar bibliotecas

# EDA
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Visualizações
import plotly.express as px

# Para preparação dos dados
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, log_loss, roc_curve, roc_auc_score, f1_score, precision_score

# Otimização de Hiperparâmetros
import optuna

### Carregar os dados

In [5]:
# Carregar o dataset
df_employees = pd.read_csv('./datasets/employee_churn.csv', parse_dates=[
  'data_contratacao',
  'data_demissao',
  'data_ultimo_feedback',
  'data_ultimo_aumento',
  'data_ultima_mudanca_cargo'
], date_format='%Y-%m-%d')

In [6]:
# Visualizar estrutura do dataset
df_employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         2000 non-null   object        
 1   idade                      2000 non-null   int64         
 2   genero                     2000 non-null   object        
 3   estado_civil               2000 non-null   object        
 4   educacao                   2000 non-null   object        
 5   regime_trabalho            2000 non-null   object        
 6   data_contratacao           2000 non-null   datetime64[ns]
 7   data_demissao              286 non-null    datetime64[ns]
 8   tipo_demissao              286 non-null    object        
 9   cargo                      2000 non-null   object        
 10  salario_atual              2000 non-null   int64         
 11  data_ultimo_feedback       2000 non-null   datetime64[ns]
 12  data_u

In [7]:
# Visualizar primeiros registros
df_employees.head()

,id,idade,genero,estado_civil,educacao,regime_trabalho,data_contratacao,data_demissao,tipo_demissao,cargo,salario_atual,data_ultimo_feedback,data_ultimo_aumento,data_ultima_mudanca_cargo,nota_avaliacao,acompanhamento_psicologo,qtde_projetos,qtde_clientes,nivel_satisfacao_gestor,churn
0,EMP1564,37,F,Divorciado(a),Master,Remoto,2020-01-02,NaT,NaN,Senior Developer,10207,2024-05-09,2023-07-31,2022-11-03,9.6,True,5,1,7.0,0
1,EMP0959,45,Outro,Divorciado(a),Master,Remoto,2020-01-03,NaT,NaN,UX Designer,23921,2024-04-05,2023-09-10,2022-02-11,8.4,True,8,4,9.1,0
2,EMP0494,54,M,Viúvo(a),Technical Degree,Híbrido,2020-01-04,NaT,NaN,Tech Lead,15298,2023-06-16,2021-12-26,2021-05-10,8.5,False,9,7,1.3,0
3,EMP1231,44,M,Casado(a),Bachelor,Remoto,2020-01-05,NaT,NaN,QA Engineer,15306,2021-12-04,2022-01-01,2024-08-30,7.9,False,8,4,8.4,0
4,EMP1912,53,Outro,Viúvo(a),PhD,Presencial,2020-01-07,NaT,NaN,Senior Developer,17723,2022-03-04,2023-11-18,2024-08-08,7.5,True,5,2,8.2,0


In [14]:
# Visualizar últimos registros
df_employees.tail()

,id,idade,genero,estado_civil,educacao,regime_trabalho,data_contratacao,data_demissao,tipo_demissao,cargo,...,nota_avaliacao,acompanhamento_psicologo,qtde_projetos,qtde_clientes,nivel_satisfacao_gestor,churn,tempo_empresa,dias_desde_ultimo_feedback,dias_desde_ultimo_aumento,dias_desde_ultima_mudanca_cargo
1995,EMP0353,50,Outro,Viúvo(a),Bachelor,Remoto,2024-07-23,NaT,NaN,QA Engineer,...,9.3,True,6,3,10.0,0,125,96,83,107
1996,EMP1808,26,F,Divorciado(a),PhD,Presencial,2024-07-24,2024-10-06,Voluntária,Senior Developer,...,9.9,False,4,6,4.7,1,74,69,69,52
1997,EMP0503,35,F,Viúvo(a),PhD,Presencial,2024-07-24,NaT,NaN,Senior Developer,...,8.0,False,6,1,6.8,0,124,42,102,37
1998,EMP1266,33,F,Viúvo(a),Technical Degree,Híbrido,2024-07-25,NaT,NaN,QA Engineer,...,8.7,False,8,1,2.6,0,123,66,110,45
1999,EMP1815,53,F,Solteiro(a),PhD,Presencial,2024-07-25,NaT,NaN,QA Engineer,...,7.9,False,9,2,2.3,0,123,47,75,82


### Engenharia de Features

In [13]:
# Criar features baseadas em datas (diferença em dias)

# Calcular tempo de empresa
df_employees['tempo_empresa'] = df_employees.apply(
  lambda x: (pd.Timestamp.now() - x['data_contratacao']).days 
  if x['churn'] == 0
  else (x['data_demissao'] - x['data_contratacao']).days, axis=1
)

# Calcular tempo desde o último feedback
df_employees['dias_desde_ultimo_feedback'] = (pd.Timestamp.now() - df_employees['data_ultimo_feedback']).dt.days

# Calcular tempo desde o último aumento
df_employees['dias_desde_ultimo_aumento'] = (pd.Timestamp.now() - df_employees['data_ultimo_aumento']).dt.days

# Calcular tempo desde a última mudança de cargo
df_employees['dias_desde_ultima_mudanca_cargo'] = (pd.Timestamp.now() - df_employees['data_ultima_mudanca_cargo']).dt.days

In [15]:
# Remover Id
df_employees.drop(columns=['id'], axis=1, inplace=True)

### EDA

In [17]:
# Verificação de valores ausentes
print("Valores ausentes por coluna:")
df_employees.isnull().sum()

Valores ausentes por coluna:


idade                                 0
genero                                0
estado_civil                          0
educacao                              0
regime_trabalho                       0
data_contratacao                      0
data_demissao                      1714
tipo_demissao                      1714
cargo                                 0
salario_atual                         0
data_ultimo_feedback                  0
data_ultimo_aumento                   0
data_ultima_mudanca_cargo             0
nota_avaliacao                        0
acompanhamento_psicologo              0
qtde_projetos                         0
qtde_clientes                         0
nivel_satisfacao_gestor               0
churn                                 0
tempo_empresa                         0
dias_desde_ultimo_feedback            0
dias_desde_ultimo_aumento             0
dias_desde_ultima_mudanca_cargo       0
dtype: int64

In [18]:
# Distribuição da variável target em percentual
fig = px.bar(
  df_employees['churn'].value_counts() / len(df_employees) * 100,
  title='Fator de Churn',
  labels={'index': 'Churn', 'value': 'Percentual'},
  opacity=0.8
)

fig.update_layout(showlegend=False)
fig.show()

In [19]:
# Valores possíveis para variáveis categóricas
for col in df_employees.select_dtypes(include=['object']).columns:
  print(f'\nValores únicos em {col}:')
  print(df_employees[col].unique())


Valores únicos em genero:
['F' 'Outro' 'M']

Valores únicos em estado_civil:
['Divorciado(a)' 'Viúvo(a)' 'Casado(a)' 'Solteiro(a)']

Valores únicos em educacao:
['Master' 'Technical Degree' 'Bachelor' 'PhD']

Valores únicos em regime_trabalho:
['Remoto' 'Híbrido' 'Presencial']

Valores únicos em tipo_demissao:
[nan 'Involuntária' 'Voluntária']

Valores únicos em cargo:
['Senior Developer' 'UX Designer' 'Tech Lead' 'QA Engineer'
 'Junior Developer' 'Product Manager' 'DevOps Engineer' 'Data Scientist']


In [20]:
# Estatísticas descritivas das variáveis numéricas
df_employees.select_dtypes(include=['int64', 'float64']).describe()

,idade,salario_atual,nota_avaliacao,qtde_projetos,qtde_clientes,nivel_satisfacao_gestor,churn,tempo_empresa,dias_desde_ultimo_feedback,dias_desde_ultimo_aumento,dias_desde_ultima_mudanca_cargo
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.000000
mean,37.938500,14801.834500,7.997550,4.984500,3.998500,5.492450,0.14300,887.686000,519.89900,533.005000,520.136500
std,9.456132,5763.451836,1.157324,2.593188,1.987076,2.609817,0.35016,492.150585,395.59628,397.623967,395.911115
min,22.000000,5016.000000,6.000000,1.000000,1.000000,1.000000,0.00000,34.000000,33.00000,34.000000,33.000000
25%,30.000000,9844.250000,7.000000,3.000000,2.000000,3.300000,0.00000,453.750000,189.75000,195.750000,193.750000
50%,38.000000,14822.500000,8.000000,5.000000,4.000000,5.500000,0.00000,880.000000,418.50000,433.000000,410.000000
75%,46.000000,19702.500000,9.000000,7.000000,6.000000,7.700000,0.00000,1308.500000,759.25000,792.250000,753.000000
max,54.000000,24988.000000,10.000000,9.000000,7.000000,10.000000,1.00000,1789.000000,1752.00000,1705.000000,1745.000000


In [21]:
# Boxplots para variáveis numéricas com Churn
for col in df_employees.select_dtypes(include=['int64', 'float64']).columns:
  if col != 'churn':
    fig = px.box(
      df_employees,
      x='churn',
      y=col,
      title=f'Boxplot de {col} vs Churn',
      color='churn'
    )
    fig.show()

In [22]:
# Matriz de Correlação
colunas_numericas = df_employees.select_dtypes(include=['int64', 'float64']).columns
corr_matrix = df_employees[colunas_numericas].corr()

fig = px.imshow(
  corr_matrix,
  title='Matriz de Correlação',
  color_continuous_scale='Viridis',
  zmin=-1,
  zmax=1
)

fig.update_traces(
  text=corr_matrix,
  texttemplate='%{text:.1%}',
  textfont={'size': 9}
)
fig.update_layout(
  width=1000,
  height=600,
  title_font=dict(size=14),
  font=dict(size=10)
)

fig.show()

In [28]:
# Matriz de dispersão
fig = px.scatter_matrix(
  df_employees,
  dimensions=colunas_numericas,
  color='churn',
  title='Matriz de Dispersão'
)

fig.update_layout(
  width=1200,
  height=1000,
  title_font=dict(size=14),
  font=dict(size=9)
)

fig.show()

In [30]:
# Teste de hipótese para variáveis categóricas vs Churn
colunas_categoricas = df_employees.select_dtypes(include=['object']).columns
for col in colunas_categoricas:
  contingency_table = pd.crosstab(df_employees['churn'], df_employees[col])
  chi2, p_value, dof, expected = chi2_contingency(contingency_table)
  print(f'\nTeste Chi-quadrado para {col} vs Churn:')
  print(f'p-valor: {p_value}')
  if p_value <= 0.05:
    print(f'As variáveis {col} e Churn não são independentes')
  else:
    print(f'As variáveis {col} e Churn são independentes')


Teste Chi-quadrado para genero vs Churn:
p-valor: 0.6391382063144609
As variáveis genero e Churn são independentes

Teste Chi-quadrado para estado_civil vs Churn:
p-valor: 0.8332629331272039
As variáveis estado_civil e Churn são independentes

Teste Chi-quadrado para educacao vs Churn:
p-valor: 0.20475149121388256
As variáveis educacao e Churn são independentes

Teste Chi-quadrado para regime_trabalho vs Churn:
p-valor: 0.9751275918764937
As variáveis regime_trabalho e Churn são independentes

Teste Chi-quadrado para tipo_demissao vs Churn:
p-valor: 1.0
As variáveis tipo_demissao e Churn são independentes

Teste Chi-quadrado para cargo vs Churn:
p-valor: 0.4586024122014404
As variáveis cargo e Churn são independentes


In [31]:
# Gráficos de barras para variáveis categóricas vs Churn
for col in colunas_categoricas:
  fig = px.histogram(
    df_employees,
    x=col,
    color='churn',
    barmode='group',
    title=f'Distribuição de {col} por Churn'
  )

  fig.show()

### Preparação dos dados

In [33]:
# Separar features e target

# Criar lista de colunas do tipo datetime
colunas_data = list(df_employees.select_dtypes(include=['datetime64']).columns)

# X e y
X = df_employees.drop(columns=['churn', 'tipo_demissao'] + colunas_data, axis=1)
y = df_employees['churn']

In [34]:
# Criar preprocessador

# Lista de Colunas por Tipo
features_numericas = X.select_dtypes(include=['int64', 'float64']).columns
features_categoricas = X.select_dtypes(include=['object']).columns

# Preprocessador
preprocessor = ColumnTransformer(
  transformers=[
    ('num', StandardScaler(), features_numericas),
    ('cat', OneHotEncoder(handle_unknown='ignore'), features_categoricas)
  ]
)

In [35]:
# Dividir os dados entre Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=51, shuffle=True)

# Aplicar o Column Transformer
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [36]:
# Mostrar os conjuntos
print(X_train.shape)
print(X_test.shape)

(1000, 32)
(1000, 32)
